<a href="https://colab.research.google.com/github/joyceee916/ATMS-597-SP-2020-Project-4/blob/master/Project4_Joyce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

In [23]:
cd /content/drive/My Drive/ATMS 597/Project4

/content/drive/My Drive/ATMS 597/Project4


Tasks
1. Use multiple linear regression to predict weather using past Global Forecast System (GFS) forecast data. 

2. Use random forest regression to predict weather using past GFS forecast data. 

Predict these variables for the period 6 UTC to 6 UTC the following day: 
* Maximum Temperature (C)
* Minimum Temperature (C)
* Maximum Wind Speed (m/s)
* Total precipitation accumulation (mm)



Day 0 at 12 UTC: you will have a model run (that's given to you)
* can incorporate observations up to but not including 00UTC 
* at the point of 00UTC will predict from 06UTC to 06UTC the next day 

Have 2 observational files (can be used in training, can also be used for verification)
* KCMI_hourly.csv
* KCMI_daily_obs.csv
* ^^ have to sum hourly observations to get total precipitation, has daily values even though the column heading says "hourly"

use pr1h or pr6h to correct for seasonal biases? (seems optional?) 

from the GFS: 
* daily.tar.gz, prof.tar.gz, sfc.tar.gz (UNIX zip files)
* for each run they contain a csv file that has the GFS forecast data 
* sfc.tar.gz - has a lot of data if you want to play around with it (sounds optional)
* prof.tar.gz - some upper air values (includes ~5 pressure levels) (can include if you want to) (sounds optional)
* each run has a timestamp that starts at 12UTC

main challenge: to get data lined up in Pandas 

start simple: start with a few variables and do a multiple linear regression, make sure things are working right 

could generate a persistence variable, try to predict tomorrow's data 

then could slowly add in more variables as you get more confidence 

could select different columns and see if your biases get better or worse as you swap out different columns - could write a function for this? 

2019 as a validation set 

calculate one metric: the RMS error (performance metric)

this week, focus on getting data read in and organized 



NOW: CAN SUBSTITUTE BICYCLE RIDERS IN WITH TEMPERATURE DATA (MODULE 4 NOTEBOOK 1): PREDICT TEMPERATURE TOMORROW 

KCMI Observational Data

In [0]:
KCMI_df = pd.read_csv("KCMI_daily_format.csv", parse_dates=True, index_col='Date')

In [0]:
KCMI_df.rename(columns={'TMAX(C)': 'Max_Temp', 'TMIN(C)': 'Min_Temp', 'WMAX(M/s)': 'Max_Wind', 'PREC(mm}': 'Total_Precip'}, inplace = True)


In [61]:
KCMI_df.head()

,Max_Temp,Min_Temp,Max_Wind,Total_Precip
Date,,,,
2010-01-01,-11.111111,-16.666667,6.70560,0.0
2010-01-02,-14.444444,-20.555556,6.70560,0.0
2010-01-03,-12.222222,-21.111111,4.91744,0.0
2010-01-04,-11.666667,-18.888889,8.94080,0.0
2010-01-05,-12.222222,-18.333333,7.15264,0.0


In [0]:
Practice_KCMI_df = KCMI_df[:]

GFS Model Data

In [0]:
GFS_df = pd.read_csv("GFS_daily_2010_2019_test.csv", parse_dates=True, index_col='Unnamed: 0')

In [29]:
GFS_df.head()

,Time,TMAX,TMIN,WMAX,RTOT
0,2010-01-01 12:00:00,-11.26,-16.46,7.323933,0.05
1,2010-01-02 12:00:00,-10.16,-15.96,4.687217,0.00
2,2010-01-03 12:00:00,-9.96,-14.36,5.941380,0.06
3,2010-01-04 12:00:00,-8.46,-12.86,5.685068,0.17
4,2010-01-05 12:00:00,-7.16,-11.06,3.794733,0.77


In [0]:
Practice_GFS_df = GFS_df[:]

Linear Regression 

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [63]:
# TODO change to the actual file (not practice)
x = Practice_KCMI_df.drop(['Total_Precip'], axis=1)
y = Practice_KCMI_df['Total_Precip']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=123)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(3286, 3) (366, 3) (3286,) (366,)


In [67]:
Practice_KCMI_df

,Max_Temp,Min_Temp,Max_Wind,Total_Precip
Date,,,,
2010-01-01,-11.111111,-16.666667,6.70560,0.0
2010-01-02,-14.444444,-20.555556,6.70560,0.0
2010-01-03,-12.222222,-21.111111,4.91744,0.0
2010-01-04,-11.666667,-18.888889,8.94080,0.0
2010-01-05,-12.222222,-18.333333,7.15264,0.0
...,...,...,...,...
2019-12-27,6.111111,0.000000,6.70560,0.0
2019-12-28,13.333333,0.000000,10.28192,2.8
2019-12-29,15.000000,5.000000,10.28192,4.3


In [69]:
Practice_KCMI_df[(Practice_KCMI_df.index == '2018-02-10')]

,Max_Temp,Min_Temp,Max_Wind,Total_Precip
Date,,,,
2018-02-10,-2.222222,-6.111111,8.04672,0.3


In [74]:
x_train[(x_train.index == '2017-06-26')]

,Max_Temp,Min_Temp,Max_Wind
Date,,,
2017-06-26,NaN,NaN,NaN


In [75]:
# Verified these null values, what to do about them 
x_train[x_train['Max_Temp'].isnull()]

,Max_Temp,Min_Temp,Max_Wind
Date,,,
2017-06-26,NaN,NaN,NaN
2014-09-24,NaN,NaN,NaN
2018-01-10,NaN,NaN,11.62304
2013-06-26,NaN,NaN,NaN
2011-05-31,NaN,NaN,11.62304
2014-09-08,NaN,NaN,NaN
2014-10-04,NaN,NaN,9.83488
2014-09-07,NaN,NaN,NaN
2014-10-03,NaN,NaN,10.72896


In [51]:
x_train.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
x_train[0:5]

,Max_Temp,Min_Temp,Max_Wind
Date,,,
2018-02-10,-2.222222,-6.111111,8.04672
2012-04-21,12.777778,4.444444,7.15264
2016-04-19,26.666667,12.222222,6.70560
2017-08-13,27.222222,13.333333,3.12928
2014-10-21,15.555556,6.111111,6.25856


In [0]:
model = LinearRegression(fit_intercept=False)

x_train= np.array(x_train['Max_Temp']).reshape(-1, 1)
y_train= np.array(y_train).reshape(-1, 1)
x_test = np.array(x_test['Max_Temp']).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

7357.200000000001

In [45]:
x = np.isnan(y_test)
print(np.argwhere(np.isnan(x)))

[]


In [34]:
lin_model = LinearRegression()
lin_model.fit(x_train, y_train)

print('Coefficients: \n', lin_model.coef_)
print('Mean squared error: %.2f' % mean_squared_error(y_test, lin_model.predict(x_test)))
print('Coefficient of determination: %.2f' % r2_score(y_test, lin_model.predict(x_test)))

plt.scatter(x_test, y_test, color='black')
plt.plot(x_test, lin_model.predict(x_test), color='blue', linewidth=3)

plt.xticks()
plt.yticks()
plt.title('Linear Regression')

ValueError: ignored

Machine Learning

Adapted from:
https://nbviewer.jupyter.org/github/srnghn/ml_example_notebooks/blob/master/Predicting%20Yacht%20Resistance%20with%20Decision%20Trees%20%26%20Random%20Forests.ipynb?source=post_page---------------------------

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# TODO change back to KCMI_daily 
x = Practice_KCMI_daily.drop(['Total_Precip'], axis=1)
y = Practice_KCMI_daily['Total_Precip']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=123)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [0]:
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor 

tree_model = DecisionTreeRegressor()
rf_model = RandomForestRegressor()

tree_model.fit(x_train, y_train) 
rf_model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

In [0]:
# TODO look up difference between mean squared error and mean absolute error 
tree_mse = mean_squared_error(y_train, tree_model.predict(x_train))
tree_mae = mean_absolute_error(y_train, tree_model.predict(x_train))
rf_mse = mean_squared_error(y_train, rf_model.predict(x_train))
rf_mae = mean_absolute_error(y_train, rf_model.predict(x_train))

In [0]:
print("Decision Tree training mse = ",tree_mse," & mae = ",tree_mae," & rmse = ", sqrt(tree_mse))
print("Random Forest training mse = ",rf_mse," & mae = ",rf_mae," & rmse = ", sqrt(rf_mse))

Decision Tree training mse =  0.000370371325425477  & mae =  0.002140278494069108  & rmse =  0.01924503378603106
Random Forest training mse =  0.0027563299475627215  & mae =  0.02149643176404135  & rmse =  0.05250076139983802


In [0]:
tree_test_mse = mean_squared_error(y_test, tree_model.predict(x_test))
tree_test_mae = mean_absolute_error(y_test, tree_model.predict(x_test))
rf_test_mse = mean_squared_error(y_test, rf_model.predict(x_test))
rf_test_mae = mean_absolute_error(y_test, rf_model.predict(x_test))

In [0]:
print("Decision Tree test mse = ",tree_test_mse," & mae = ",tree_test_mae," & rmse = ", sqrt(tree_test_mse))
print("Random Forest test mse = ",rf_test_mse," & mae = ",rf_test_mae," & rmse = ", sqrt(rf_test_mse))

Decision Tree test mse =  0.04007824074074074  & mae =  0.06689814814814815  & rmse =  0.20019550629507332
Random Forest test mse =  0.024299461099280377  & mae =  0.06208412477954146  & rmse =  0.15588284414675135


Tar Files

In [0]:
Daily_Tar = open("daily.tar.gz")